In [ ]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import geopandas
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image
import sys
import os

In [ ]:
scyjava.config.add_option('-Xmx30g')
#ij = imagej.init('/home/saka/fiji-linux64/Fiji.app', mode = 'interactive')
ij = imagej.init('/home/saka/sw/local/fiji/2023', mode='interactive')
ij.ui().showUI()
ij.getVersion()

In [ ]:
showPolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Overlay = scyjava.jimport('ij.gui.Overlay')
Regions = scyjava.jimport('net.imglib2.roi.Regions')
LabelRegions = scyjava.jimport('net.imglib2.roi.labeling.LabelRegions')
ZProjector = scyjava.jimport('ij.plugin.ZProjector')()
Duplicator = scyjava.jimport('ij.plugin.Duplicator')()
ov = Overlay()
rm = ij.RoiManager.getRoiManager()

In [ ]:
root = tk.Tk()
root.withdraw()
directory_path = filedialog.askdirectory()

In [ ]:
# Get a list of all the folders in the directory
folder_list = os.listdir(directory_path)
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)
    # Find all the ".czi" files in the folder
    file_pattern = os.path.join(folder_path, "*.czi")
    file_list = glob.glob(file_pattern)
    for file_path in file_list:
        image = ij.io().open(file_path)
        wanted_channel = 0
        #wanted_z = 2
        wanted_image = image[:, :, wanted_channel]
        result_path = os.path.splitext(file_path)[0] + "_blue.tif"
        cellpose_image = ij.py.to_dataset(wanted_image)
        ij.io().save(cellpose_image, result_path)
        print(f"Saving image {result_path}.")

In [ ]:
model = models.CellposeModel(gpu=True, model_type='ligma')
files = get_image_files(directory_path, '_masks', look_one_level_down=True)
imgs = [imread(f) for f in files]
nimg = len(imgs)
channels = [[0,0]]
masks, flows, styles = model.eval(imgs, diameter=None, channels=channels)
io.save_to_png(imgs, masks, flows, files)

In [ ]:
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)

    # Find all the z-projected nuclear files in the folder
    file_pattern = os.path.join(folder_path, "*_blue.tif")
    file_list = glob.glob(file_pattern)
    
    for file_path in file_list:
        image = ij.io().open(file_path)
        f_name = os.path.basename(file_path)
        f_name = os.path.splitext(f_name)[0]
        input_txt = Path(f"{folder_path}/{f_name}_cp_outlines.txt")
        txt_fh = open(input_txt, 'r')
        #ij.ui().show(image)
        #to better draw, apply and save ROIs, the image should be displayed:
        imp = ij.py.to_imageplus(image) 
        #Call of .txt cellpose output file and draw ROIs based on segmentation result, and add them in the ROI manager
        for line in txt_fh:
            xy = line.rstrip().split(",")
            xy_coords = [int(element) for element in xy if element not in '']
            x_coords = [int(element) for element in xy[::2] if element not in '']
            y_coords = [int(element) for element in xy[1::2] if element not in '']
            xcoords_jint = JArray(JInt)(x_coords)
            ycoords_jint = JArray(JInt)(y_coords)
            polygon_roi_instance = scyjava.jimport('ij.gui.PolygonRoi')
            roi_instance = scyjava.jimport('ij.gui.Roi')
            imported_polygon = polygon_roi_instance(xcoords_jint, ycoords_jint, len(x_coords), int(roi_instance.POLYGON))
            imp.setRoi(imported_polygon)
            rm.addRoi(imported_polygon)
        ij.py.run_macro("roiManager('Select All');")
        rm.runCommand("Save", f"{folder_path}/" + f"{f_name}.zip")
        ij.py.run_macro("roiManager('Select All');")
        rm.runCommand("Delete")

In [ ]:
# Get a list of all the folders in the directory
folder_list = os.listdir(directory_path)
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)
    # Find all the ".czi" files in the folder
    file_pattern = os.path.join(folder_path, "*.czi")
    file_list = glob.glob(file_pattern)
    for file_path in file_list:
        image = ij.io().open(file_path)
        wanted_channel = 1
        #wanted_z = 2
        wanted_image = image[:, :, wanted_channel]
        result_path = os.path.splitext(file_path)[0] + "_red.tif"
        cellpose_image = ij.py.to_dataset(wanted_image)
        ij.io().save(cellpose_image, result_path)
        print(f"Saving image {result_path}.")

In [ ]:
folder_list = os.listdir(directory_path)
# Iterate over the folders and process each folder
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)
    file_prefix = "Snap-"  # Specify the common prefix of the files. Change it as your basic file is formatted.

# Regular expression pattern to match the number in the filename
    pattern = re.compile(rf"{file_prefix}(\d+)_blue\.zip", re.IGNORECASE)

# Iterate over the files in the folder
    for filename in os.listdir(folder_path):
    # Check if the file matches the desired pattern
        match = pattern.match(filename)
        if match:
        # Extract the number from the filename
            current_number = int(match.group(1))
        # Construct the new filename with the updated number
            new_filename = f"{file_prefix}{current_number}_red.zip"
        # Create the full file paths for the old and new filenames
            old_filepath = os.path.join(folder_path, filename)
            new_filepath = os.path.join(folder_path, new_filename)
        # Rename the file
            os.rename(old_filepath, new_filepath)
            print(f"Renamed file: {filename} -> {new_filename}")

In [18]:
set_string = f'Set Measurements...'
measure_string = f'mean integrated redirect=None decimal=2'
ij.IJ.run(set_string, measure_string)
for folder in folder_list:
    # Get the path to the folder
    folder_path = os.path.join(directory_path, folder)
    # Find all the ".czi" files in the folder
    file_pattern = os.path.join(folder_path, "*_red.tif")
    file_list = glob.glob(file_pattern)
    for file_path in file_list:
        image = ij.io().open(file_path)
        ij.ui().show(image)
        f_name = os.path.basename(file_path)
        basename = os.path.splitext(f_name)[0]
        input_roi = Path(f"{folder_path}/{basename}.zip")
        rm.open(f"{input_roi}")
        roi_number = rm.getCount()
        for roi in range(roi_number):
            rm.select(roi-1)
            ij.IJ.run('Measure')
        ij.py.run_macro("roiManager('Select All');")
        rm.runCommand("Delete")
        ij.IJ.selectWindow(f"{f_name}")
        ij.IJ.run('Close')
    folder_name = os.path.basename(folder_path)
    output = Path(f"{directory_path}/{folder_name}.csv").as_posix()
    saving = ij.IJ.saveAs("Results", output)
    ij.IJ.run("Clear Results")
            
        